In [369]:
from lusidtools.jupyter_tools import toggle_code

"""Equity - Handeling Corporate Actions

Attributes
----------
equity
split
merger
spin-off
bonus issue
dividend
recipes
valuations
corporate actions
"""

toggle_code("Toggle Docstring")

# Creation and Lifecycle of Corporate Actions

In this notebook, we demonstrate how you can create corporate actions and apply them to a portfolio.
We will also demonstrate how the holdings in the portfolio change as we go through the lifecycle of the corporate action.

## Table of Contents:
* [1. Upload Instrument Master](#1.-Upload-Instrument-Master)
* [2. Create a Corporate Action Source and Link to a Portfolio](#2.-Create-a-Corporate-Action-Source-and-Link-to-a-Portfolio)
* [3. Transactions](#3.-Transactions)
* [4. Creating a Corporate Action](#4.-Creating-a-Corporate-Action)
* [4.1 Dividend](#4.1-Dividend)
* [4.2 Merger](#4.2-Merger)
* [4.3 Spin-off](#4.3-Spin-off)
* [4.4 Split](#4.4-Split)
* [4.5 Bonus Issue](#4.5-Bonus-Issue)

In [370]:
# Import generic non-LUSID packages
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json
import pytz
from IPython.core.display import HTML

# Import key modules from the LUSID package
import lusid as lu
import lusid.models as lm

# Import key functions from Lusid-Python-Tools and other packages
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidjam import RefreshingToken
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_transactions_response,
)


# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

# display(HTML("<style>.container { width:90% !important; }</style>"))

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate our user and create our API client
api_factory = lu.utilities.ApiClientFactory(
    token=RefreshingToken(), api_secrets_filename=secrets_path
)

print("LUSID Environment Initialised")
print(
    "LUSID API Version :",
    api_factory.build(lu.api.ApplicationMetadataApi).get_lusid_versions().build_version,
)

LUSID Environment Initialised
LUSID API Version : 0.6.8916.0


We will first set up our APIs and variables as well as create 2 helper functions that we will use in this notebook.

In [371]:
# LUSID Variable Definitions
portfolio_api = api_factory.build(lu.api.PortfoliosApi)
transaction_portfolios_api = api_factory.build(lu.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lu.api.InstrumentsApi)
corporate_action_sources_api = api_factory.build(lu.api.CorporateActionSourcesApi)

In [372]:
# Define scopes
scope = "ibor"
ca_code = "ibor_corp_act"
portfolio_code = "CorporateActionsPortfolio"

In [373]:
def to_datetime(date):
    datetime_date = datetime.strptime(date, "%d/%m/%Y").replace(tzinfo=pytz.utc)
    return datetime_date

def get_portfolio_holdings(date):
    holdings=transaction_portfolios_api.get_holdings(
        scope=scope,
        code=portfolio_code,
        effective_at=to_datetime(date),
        property_keys=['instrument/default/Name']
        )

    return lusid_response_to_data_frame(holdings)[['properties.Instrument/default/Name.value.label_value',"units", "settled_units",	"cost.amount", "cost.currency", "cost_portfolio_ccy.amount"]]

## 1. Upload instrument master

Before creating a portfolio, we must first define the assets that we will be working with by uploading an instrument master.

In [374]:
instruments = pd.read_csv('data/corporateactions-instruments.csv')
instruments

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,nan
1,Apple_Nasdaq_AAPL,imd_35345345,USD,US0378331005,BBG000B9XVV8,UN,united_states_america,AAPL,equity,common_stock,nan
2,ExpressScripts_NYSE_ESRX,imd_34352311,USD,US30219G1085,BBG000C16621,UN,united_states_america,ESRX,equity,common_stock,nan
3,TrinityIndustries_NYSE_TRN,imd_34235200,USD,US8965221091,BBG000BVL406,UN,united_states_america,TRN,equity,common_stock,nan
4,Trex_NYSE_TREX,imd_32423956,USD,US89531P1057,BBG000BTGM43,UN,united_states_america,TREX,equity,common_stock,nan
5,Cigna_NYSE_CI,imd_32452391,USD,US1255091092,BBG00KXXK940,UN,united_states_america,CI,equity,common_stock,nan
6,Arcosa_NYSE_ACA,imd_23423409,USD,US0396531008,BBG00JGMWFQ5,UN,united_states_america,ACA,equity,common_stock,nan
7,ACM_Research_Inc,imd_23423410,USD,US0236530000,BBG00HPSG933,UN,united_states_america,ACMR,equity,common_stock,nan


In [375]:
# Load the instrument into LUSID

instrument_identifier_mapping = {
    "Figi": "figi",
    "Isin": "isin",
}

instrument_mapping_required = {"name": "instrument_name"}
instrument_mapping_optional = {}

responses = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=instruments,
    mapping_required=instrument_mapping_required,
    mapping_optional=instrument_mapping_optional,
    file_type="instrument",
    identifier_mapping=instrument_identifier_mapping,
)

succ, failed, errors = format_instruments_response(responses)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

,success,failed,errors
0,8,0,0


## 2. Create a Corporate Action Source and Link to a Portfolio

All corporate actions are stored in a corporate actions source. This source will then be linked to a portfolio using the corporate_action_source_id parameter upon creation of the portfolio. Alternatively, you can also update the corporate actions source of a portfolio after creating it by using https://www.lusid.com/docs/api/#operation/UpsertPortfolioDetails.

In [376]:
try:

    source_request = lm.CreateCorporateActionSourceRequest(
        scope=scope,
        code=ca_code,
        display_name="Ibor Corporate Action Source",
        description="Corporate Actions source for sample notebook",
    )

    source_result = api_factory.build(
        lu.api.CorporateActionSourcesApi
    ).create_corporate_action_source(
        create_corporate_action_source_request=source_request
    )

except lu.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a Corporate Action Source with id ibor_corp_act because it already exists in scope ibor.


In [377]:
try:
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="USD",
            created="2010-01-01",
            sub_holding_keys=[],
            corporate_action_source_id=lu.ResourceId(scope=scope, code=ca_code),
        ),
    )

except lu.ApiException as e:
    print(json.loads(e.body)["title"])

## 3. Transactions

We populate our portfolio with transactions so we have assets to perform our corporate actions on.

In [378]:
# upsert transactions
transactions = pd.read_csv("data/corporate_actions_transactions.csv")
transactions

,portfolio_code,transaction_id,figi,ticker,name,transaction_type,transaction_date,settlement_date,transaction_units,transaction_price,total_consideration,transaction_currency
0,CorporateActionsPortfolio,TX001,BBG000B9XVV8,AAPL,Apple_Nasdaq_AAPL,StockIn,01/01/2018,02/01/2018,100,43.75,"4,375.00",USD
1,CorporateActionsPortfolio,TX002,BBG000C16621,ESRX,ExpressScripts_NYSE_ESRX,StockIn,01/01/2018,02/01/2018,100,76.01,"7,601.00",USD
2,CorporateActionsPortfolio,TX003,BBG000BVL406,TRN,TrinityIndustries_NYSE_TRN,StockIn,01/01/2018,02/01/2018,100,26.34,"2,634.00",USD
3,CorporateActionsPortfolio,TX004,BBG000BTGM43,TREX,Trex_NYSE_TREX,StockIn,01/01/2018,02/01/2018,100,27.83,"2,783.00",USD
4,CorporateActionsPortfolio,TX005,BBG00HPSG933,ACMR,ACM_Research_Inc,StockIn,01/01/2018,02/01/2018,100,2.00,200.00,USD


In [379]:
# map the transactions file

transaction_field_mapping_required = {
    "code": "portfolio_code",
    "transaction_id": "transaction_id",
    "type": "transaction_type",
    "transaction_date": "transaction_date",
    "settlement_date": "settlement_date",
    "units": "transaction_units",
    "transaction_price.price": "transaction_price",
    "transaction_price.type": "$Price",
    "total_consideration.amount": "total_consideration",
    "total_consideration.currency": "transaction_currency",
    "transaction_currency": "transaction_currency",
}


transaction_identifier_mapping = {
    "Figi": "figi",
}

In [380]:
responses = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=transactions,
    mapping_required=transaction_field_mapping_required,
    mapping_optional={},
    identifier_mapping=transaction_identifier_mapping,
    file_type="transaction",
)

succ, failed = format_transactions_response(responses)
display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed)}]))

,success,failed
0,1,0


## 4. Creating corporate actions

To generate our corporate actions, we have a file that contains the different corporate action dates as well as the input and output factors of the assets. You can use the table below for reference and we will be taking out rows of this table to create our corporate actions.

In [381]:
ca_data = pd.read_csv("data/corporate_actions.csv")
ca_data

,Description,code,action_description,announcement_date,ex_date,record_date,payment_date,input_instrument_figi,instrument_input_ticker,input_instrument_name,input_units_factor,input_cost_factor,output_instrument_figi,output_instrument_name,output_ticker,output_instrument_internal,output_units_factor,output_cost_factor
0,Apple Dividend of 0.17 cents per share,ca001,dividend,29/01/2019,08/02/2019,11/02/2019,14/02/2019,BBG000B9XVV8,AAPL,APPLE INC,1,1,NaN,NaN,NaN,CCY_USD,0.02,0
1,Cigna & Express Scripts Merger,ca002,merger (equivalent to exchange example),08/03/2018,10/12/2018,13/12/2018,20/12/2018,BBG000C16621,ESRX,EXPRESS SCRIPTS HOLDINGS CO,1,1,BBG00KXXK940,CIGNA CORP,CI,NaN,0.24,1
2,Acosa spin-off from Trinity Industries,ca003,spin-off (equivalent to exchange example),01/01/2018,17/10/2018,20/10/2018,01/11/2018,BBG000BVL406,TRN,TRINITY INDUSTRIES INC,3,1,BBG00JGMWFQ5,ARCOSA INC,ACA,NaN,1.00,0
3,Trex 2 for 1 stock split,ca004,split,07/05/2018,21/05/2018,23/05/2018,19/06/2018,BBG000BTGM43,TREX,TREX COMPANY INC,1,1,BBG000BTGM43,TREX COMPANY INC,TREX,NaN,2.00,1
4,ACM Research Inc bonus issue,ca005,bonus issue,03/03/2022,14/03/2022,16/03/2022,23/03/2022,BBG00HPSG933,ACMR,ACM Research Inc,1,1,BBG00HPSG933,ACM Research Inc,ACMR,NaN,2.00,1


### 4.1. Dividend

A dividend will give us a certain amount of money or shares for every share we currently hold in a company. In our scenario, we are getting 17 cents per share of Apple stock. We own 100 shares in Apple so we can expect to receive 1.70 USD. We can see that our input factor is 1 and our output factor is 0.017 (this is displayed as 0.02 in the above table due to rounding). This means that for every 1 unit of our input instrument (Apple shares) we will receive 0.017 units of our output instrument (USD).

In [382]:
dataset = ca_data.iloc[0]

transitions = [
    lm.CorporateActionTransition(
        input_transition=lm.CorporateActionTransitionComponentRequest(
            instrument_identifiers={
                "Instrument/default/Figi": dataset['input_instrument_figi'],
            },
            units_factor=float(dataset['input_units_factor']),
            cost_factor=float(dataset['input_cost_factor']),
        ),
        output_transitions=[
            lm.CorporateActionTransitionComponentRequest(
                instrument_identifiers={"Instrument/default/Currency": "USD"},
                units_factor=float(dataset['output_units_factor']),
                cost_factor=float(dataset['output_cost_factor']),
            )
        ],
    )
]

dividend_request = lm.UpsertCorporateActionRequest(
    corporate_action_code=dataset['code'],
    announcement_date=to_datetime(dataset['announcement_date']),
    ex_date=to_datetime(dataset['ex_date']),
    record_date=to_datetime(dataset['record_date']),
    payment_date=to_datetime(dataset['payment_date']),
    transitions=transitions,
)

corporate_action_sources_api.batch_upsert_corporate_actions(
    scope=scope, code=ca_code, upsert_corporate_action_request=[dividend_request]
)

{'failed': {},
 'href': None,
 'links': [{'description': None,
            'href': 'https://lorenz.lusid.com/api/api/schemas/entities/UpsertCorporateActionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://lorenz.lusid.com/app/insights/logs/0HMGJ5N8F49BT:0000001D',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'ca001': {'announcement_date': datetime.datetime(2019, 1, 29, 0, 0, tzinfo=tzutc()),
                      'corporate_action_code': 'ca001',
                      'description': None,
                      'ex_date': datetime.datetime(2019, 2, 8, 0, 0, tzinfo=tzutc()),
                      'payment_date': datetime.datetime(2019, 2, 14, 0, 0, tzinfo=tzutc()),
                      'record_date': datetime.datetime(2019, 2, 11, 0, 0, tzinfo=tzutc()),
  

On the announcement date of the dividend, we take a look at our holdings and see just a position in Apple of 100 units.

In [383]:
get_portfolio_holdings("29/01/2019")

,properties.Instrument/default/Name.value.label_value,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,Apple_Nasdaq_AAPL,100.00,100.00,"4,375.00",USD,"4,375.00"
1,ExpressScripts_NYSE_ESRX,100.00,100.00,"7,601.00",USD,"7,601.00"
2,TrinityIndustries_NYSE_TRN,100.00,100.00,"2,634.00",USD,"2,634.00"
3,Trex_NYSE_TREX,200.00,200.00,"2,783.00",USD,"2,783.00"
4,ACM_Research_Inc,100.00,100.00,200.00,USD,200.00
5,Arcosa_NYSE_ACA,33.00,33.00,0.00,USD,0.00
6,Cigna_NYSE_CI,24.00,24.00,"7,601.00",USD,"7,601.00"


On the ex-date, we can see the USD cash dividend coming in but it has not settled yet.

In [384]:
get_portfolio_holdings("08/02/2019")

,properties.Instrument/default/Name.value.label_value,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,Apple_Nasdaq_AAPL,100.00,100.00,"4,375.00",USD,"4,375.00"
1,ExpressScripts_NYSE_ESRX,100.00,100.00,"7,601.00",USD,"7,601.00"
2,TrinityIndustries_NYSE_TRN,100.00,100.00,"2,634.00",USD,"2,634.00"
3,Trex_NYSE_TREX,200.00,200.00,"2,783.00",USD,"2,783.00"
4,ACM_Research_Inc,100.00,100.00,200.00,USD,200.00
5,Arcosa_NYSE_ACA,33.00,33.00,0.00,USD,0.00
6,Cigna_NYSE_CI,24.00,24.00,"7,601.00",USD,"7,601.00"
7,USD,1.70,0.00,1.70,USD,1.70


Finally on the payment date, we can see the USD cash position settled into our holdings.

In [385]:
get_portfolio_holdings("14/02/2019")

,properties.Instrument/default/Name.value.label_value,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,Apple_Nasdaq_AAPL,100.00,100.00,"4,375.00",USD,"4,375.00"
1,ExpressScripts_NYSE_ESRX,100.00,100.00,"7,601.00",USD,"7,601.00"
2,TrinityIndustries_NYSE_TRN,100.00,100.00,"2,634.00",USD,"2,634.00"
3,Trex_NYSE_TREX,200.00,200.00,"2,783.00",USD,"2,783.00"
4,ACM_Research_Inc,100.00,100.00,200.00,USD,200.00
5,Arcosa_NYSE_ACA,33.00,33.00,0.00,USD,0.00
6,Cigna_NYSE_CI,24.00,24.00,"7,601.00",USD,"7,601.00"
7,USD,1.70,1.70,1.70,USD,1.70


### 4.2. Merger

In our example of the merger here, we will show you what happened to Express Scripts when it got acquired by Cigna. In this merger, the shareholders of Express Scripts received 0.24 shares in Cigna for every Express Scripts share they held. We can see that the output factor for this transaction is 0.24. Given that we owned 100 shares in Express Scripts at the time of this merger, we should receive 24 shares in Cigna.

In [386]:
dataset = ca_data.iloc[1]

transitions = [
    lm.CorporateActionTransition(
        input_transition=lm.CorporateActionTransitionComponentRequest(
            instrument_identifiers={
                "Instrument/default/Figi": dataset['input_instrument_figi']
            },
            units_factor=float(dataset['input_units_factor']),
            cost_factor=float(dataset['input_cost_factor']),
        ),
        output_transitions=[
            lm.CorporateActionTransitionComponentRequest(
                instrument_identifiers={
                    "Instrument/default/Figi": dataset['output_instrument_figi']
                },
                units_factor=float(dataset['output_units_factor']),
                cost_factor=float(dataset['output_cost_factor']),
            )
        ],
    )
]

merger_request = lm.UpsertCorporateActionRequest(
    corporate_action_code=dataset['code'],
    announcement_date=to_datetime(dataset['announcement_date']),
    ex_date=to_datetime(dataset['ex_date']),
    record_date=to_datetime(dataset['record_date']),
    payment_date=to_datetime(dataset['payment_date']),
    transitions=transitions,
)

corporate_action_sources_api.batch_upsert_corporate_actions(
    scope=scope, code=ca_code, upsert_corporate_action_request=[merger_request]
)

{'failed': {},
 'href': None,
 'links': [{'description': None,
            'href': 'https://lorenz.lusid.com/api/api/schemas/entities/UpsertCorporateActionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://lorenz.lusid.com/app/insights/logs/0HMGJ5N8F49BT:0000001F',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'ca002': {'announcement_date': datetime.datetime(2018, 3, 8, 0, 0, tzinfo=tzutc()),
                      'corporate_action_code': 'ca002',
                      'description': None,
                      'ex_date': datetime.datetime(2018, 12, 10, 0, 0, tzinfo=tzutc()),
                      'payment_date': datetime.datetime(2018, 12, 20, 0, 0, tzinfo=tzutc()),
                      'record_date': datetime.datetime(2018, 12, 13, 0, 0, tzinfo=tzutc()),

On the announcement date of the merger, we can see a position of 100 units in Express Scripts in our holdings. We know that the output factor will be 0.24 so we expect 24 shares in Cigna since we hold 100 shares in Express Scripts.

In [387]:
get_portfolio_holdings("08/03/2018")

,properties.Instrument/default/Name.value.label_value,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,Apple_Nasdaq_AAPL,100.00,100.00,"4,375.00",USD,"4,375.00"
1,ExpressScripts_NYSE_ESRX,100.00,100.00,"7,601.00",USD,"7,601.00"
2,TrinityIndustries_NYSE_TRN,100.00,100.00,"2,634.00",USD,"2,634.00"
3,Trex_NYSE_TREX,100.00,100.00,"2,783.00",USD,"2,783.00"
4,ACM_Research_Inc,100.00,100.00,200.00,USD,200.00


On the ex-date, we see an unsettled position of 24 units in Cigna as expected. These will then settle on 20/12/2018 which is the payment date.

In [388]:
get_portfolio_holdings("20/12/2018")

,properties.Instrument/default/Name.value.label_value,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,Apple_Nasdaq_AAPL,100.00,100.00,"4,375.00",USD,"4,375.00"
1,ExpressScripts_NYSE_ESRX,100.00,100.00,"7,601.00",USD,"7,601.00"
2,TrinityIndustries_NYSE_TRN,100.00,100.00,"2,634.00",USD,"2,634.00"
3,Trex_NYSE_TREX,200.00,200.00,"2,783.00",USD,"2,783.00"
4,ACM_Research_Inc,100.00,100.00,200.00,USD,200.00
5,Arcosa_NYSE_ACA,33.00,33.00,0.00,USD,0.00
6,Cigna_NYSE_CI,24.00,24.00,"7,601.00",USD,"7,601.00"


### 4.3. Spin-off

In a spin-off the company will sell off a part of their business. In our example, the company will reward its shareholders with shares of the spun-off company. Owners of Trinity Industries shares will receive 1 share in Arcosa for every 3 shares they hold in Trinity Industries.

In [389]:
dataset = ca_data.iloc[2]

transitions = [
    lm.CorporateActionTransition(
        input_transition=lm.CorporateActionTransitionComponentRequest(
            instrument_identifiers={
                "Instrument/default/Figi": dataset['input_instrument_figi']
            },
            units_factor=float(dataset['input_units_factor']),
            cost_factor=float(dataset['input_cost_factor']),
        ),
        output_transitions=[
            lm.CorporateActionTransitionComponentRequest(
                instrument_identifiers={
                    "Instrument/default/Figi": dataset['output_instrument_figi']
                },
                units_factor=float(dataset['output_units_factor']),
                cost_factor=float(dataset['output_cost_factor']),
            )
        ],
    )
]

spin_off_request = lm.UpsertCorporateActionRequest(
    corporate_action_code=dataset['code'],
    announcement_date=to_datetime(dataset['announcement_date']),
    ex_date=to_datetime(dataset['ex_date']),
    record_date=to_datetime(dataset['record_date']),
    payment_date=to_datetime(dataset['payment_date']),
    transitions=transitions,
)

corporate_action_sources_api.batch_upsert_corporate_actions(
    scope=scope, code=ca_code, upsert_corporate_action_request=[spin_off_request]
)

{'failed': {},
 'href': None,
 'links': [{'description': None,
            'href': 'https://lorenz.lusid.com/api/api/schemas/entities/UpsertCorporateActionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://lorenz.lusid.com/app/insights/logs/0HMGJ2GOD7VT7:00000006',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'ca003': {'announcement_date': datetime.datetime(2018, 1, 1, 0, 0, tzinfo=tzutc()),
                      'corporate_action_code': 'ca003',
                      'description': None,
                      'ex_date': datetime.datetime(2018, 10, 17, 0, 0, tzinfo=tzutc()),
                      'payment_date': datetime.datetime(2018, 11, 1, 0, 0, tzinfo=tzutc()),
                      'record_date': datetime.datetime(2018, 10, 20, 0, 0, tzinfo=tzutc()),


On the day of the announcement, we can see that we only hold 100 shares in Trinity Industries.

In [390]:
get_portfolio_holdings("01/01/2018")

,properties.Instrument/default/Name.value.label_value,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,Apple_Nasdaq_AAPL,100.00,0.00,"4,375.00",USD,"4,375.00"
1,ExpressScripts_NYSE_ESRX,100.00,0.00,"7,601.00",USD,"7,601.00"
2,TrinityIndustries_NYSE_TRN,100.00,0.00,"2,634.00",USD,"2,634.00"
3,Trex_NYSE_TREX,100.00,0.00,"2,783.00",USD,"2,783.00"
4,ACM_Research_Inc,100.00,0.00,200.00,USD,200.00


In this spin-off, we have an input units factor of 3 and an output units factor of 1. This means that for every 3 shares in Trinity Industries we hold, we will receive 1 share in Arcosa.
We can see this reflected in our holdings, as we had 100 units of Trinity Industries and we thus we receive 33 shares in Arcosa on the payment date.

In [391]:
get_portfolio_holdings("01/11/2018")

,properties.Instrument/default/Name.value.label_value,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,Apple_Nasdaq_AAPL,100.00,100.00,"4,375.00",USD,"4,375.00"
1,ExpressScripts_NYSE_ESRX,100.00,100.00,"7,601.00",USD,"7,601.00"
2,TrinityIndustries_NYSE_TRN,100.00,100.00,"2,634.00",USD,"2,634.00"
3,Trex_NYSE_TREX,200.00,200.00,"2,783.00",USD,"2,783.00"
4,ACM_Research_Inc,100.00,100.00,200.00,USD,200.00
5,Arcosa_NYSE_ACA,33.00,33.00,0.00,USD,0.00


### 4.4. Split

During a stock split, the company will multiply the amount of shares and divide the price by an equal amount. This causes them to lower the share price but keep the market cap of the company equal. In our example we have a 2 for 1 stock split, meaning shareholders get 2 shares for every share they own and their cost basis remains the same. In this scenario our input factor is 1 and our output factor is 2 while the cost factors remain 1 for both. As a result you get 2 times the output instrument for every 1 unit of input instrument.

In [392]:
dataset = ca_data.iloc[3]

transitions = [
    lm.CorporateActionTransition(
        input_transition=lm.CorporateActionTransitionComponentRequest(
            instrument_identifiers={
                "Instrument/default/Figi": dataset['input_instrument_figi']
            },
            units_factor=float(dataset['input_units_factor']),
            cost_factor=float(dataset['input_cost_factor']),
        ),
        output_transitions=[
            lm.CorporateActionTransitionComponentRequest(
                instrument_identifiers={
                    "Instrument/default/Figi": dataset['output_instrument_figi']
                },
                units_factor=float(dataset['output_units_factor']),
                cost_factor=float(dataset['output_cost_factor']),
            )
        ],
    )
]

split_request = lm.UpsertCorporateActionRequest(
    corporate_action_code=dataset['code'],
    announcement_date=to_datetime(dataset['announcement_date']),
    ex_date=to_datetime(dataset['ex_date']),
    record_date=to_datetime(dataset['record_date']),
    payment_date=to_datetime(dataset['payment_date']),
    transitions=transitions,
)

corporate_action_sources_api.batch_upsert_corporate_actions(
    scope=scope, code=ca_code, upsert_corporate_action_request=[split_request]
)

{'failed': {},
 'href': None,
 'links': [{'description': None,
            'href': 'https://lorenz.lusid.com/api/api/schemas/entities/UpsertCorporateActionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://lorenz.lusid.com/app/insights/logs/0HMGJ1RDFFGVV:00000014',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'ca004': {'announcement_date': datetime.datetime(2018, 5, 7, 0, 0, tzinfo=tzutc()),
                      'corporate_action_code': 'ca004',
                      'description': None,
                      'ex_date': datetime.datetime(2018, 5, 21, 0, 0, tzinfo=tzutc()),
                      'payment_date': datetime.datetime(2018, 6, 19, 0, 0, tzinfo=tzutc()),
                      'record_date': datetime.datetime(2018, 5, 23, 0, 0, tzinfo=tzutc()),
  

On the announcement date of this split, we can see that we have 100 shares in Trex. The split will be 2 for 1 so we expect to have 200 shares on the payment date.

In [393]:
get_portfolio_holdings("07/05/2018")

,properties.Instrument/default/Name.value.label_value,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,Apple_Nasdaq_AAPL,100.00,100.00,"4,375.00",USD,"4,375.00"
1,ExpressScripts_NYSE_ESRX,100.00,100.00,"7,601.00",USD,"7,601.00"
2,TrinityIndustries_NYSE_TRN,100.00,100.00,"2,634.00",USD,"2,634.00"
3,Trex_NYSE_TREX,100.00,100.00,"2,783.00",USD,"2,783.00"
4,ACM_Research_Inc,100.00,100.00,200.00,USD,200.00


On the payment date, we can see that we have 200 settled units in Trex as we expected and our cost remains unaffected.

In [394]:
get_portfolio_holdings("19/06/2018")

,properties.Instrument/default/Name.value.label_value,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,Apple_Nasdaq_AAPL,100.00,100.00,"4,375.00",USD,"4,375.00"
1,ExpressScripts_NYSE_ESRX,100.00,100.00,"7,601.00",USD,"7,601.00"
2,TrinityIndustries_NYSE_TRN,100.00,100.00,"2,634.00",USD,"2,634.00"
3,Trex_NYSE_TREX,200.00,200.00,"2,783.00",USD,"2,783.00"
4,ACM_Research_Inc,100.00,100.00,200.00,USD,200.00


### 4.5. Bonus Issue

A bonus issue is essentially a stock dividend, the company will reward shareholders by handing out extra shares to current shareholders. In our example the company ACM Research will reward each shareholder with an additional share. This results in multiplying their holdings by 2. We thus have an input factor of 1 and an output factor of 2 while keeping the cost equal. This means that for every 1 share, we receive 2 shares. This will double the position and not affect the cost basis as our output cost factor is 1.

In [395]:
dataset = ca_data.iloc[4]

transitions = [
    lm.CorporateActionTransition(
        input_transition=lm.CorporateActionTransitionComponentRequest(
            instrument_identifiers={
                "Instrument/default/Figi": dataset['input_instrument_figi']
            },
            units_factor=float(dataset['input_units_factor']),
            cost_factor=float(dataset['input_cost_factor']),
        ),
        output_transitions=[
            lm.CorporateActionTransitionComponentRequest(
                instrument_identifiers={
                    "Instrument/default/Figi": dataset['output_instrument_figi']
                },
                units_factor=float(dataset['output_units_factor']),
                cost_factor=float(dataset['output_cost_factor']),
            )
        ],
    )
]

bonus_issue_request = lm.UpsertCorporateActionRequest(
    corporate_action_code=dataset['code'],
    announcement_date=to_datetime(dataset['announcement_date']),
    ex_date=to_datetime(dataset['ex_date']),
    record_date=to_datetime(dataset['record_date']),
    payment_date=to_datetime(dataset['payment_date']),
    transitions=transitions,
)

corporate_action_sources_api.batch_upsert_corporate_actions(
    scope=scope, code=ca_code, upsert_corporate_action_request=[bonus_issue_request]
)

{'failed': {},
 'href': None,
 'links': [{'description': None,
            'href': 'https://lorenz.lusid.com/api/api/schemas/entities/UpsertCorporateActionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://lorenz.lusid.com/app/insights/logs/0HMGJ5N9S3F31:0000004A',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'ca005': {'announcement_date': datetime.datetime(2022, 3, 3, 0, 0, tzinfo=tzutc()),
                      'corporate_action_code': 'ca005',
                      'description': None,
                      'ex_date': datetime.datetime(2022, 3, 14, 0, 0, tzinfo=tzutc()),
                      'payment_date': datetime.datetime(2022, 3, 23, 0, 0, tzinfo=tzutc()),
                      'record_date': datetime.datetime(2022, 3, 16, 0, 0, tzinfo=tzutc()),
  

During a bonus issue, a company will provide shareholders with extra shares in the company. On the announcement date we can see that we hold 100 shares in ACM Research.

In [396]:
get_portfolio_holdings("03/03/2022")

,properties.Instrument/default/Name.value.label_value,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,Apple_Nasdaq_AAPL,100.00,100.00,"4,375.00",USD,"4,375.00"
1,ExpressScripts_NYSE_ESRX,100.00,100.00,"7,601.00",USD,"7,601.00"
2,TrinityIndustries_NYSE_TRN,100.00,100.00,"2,634.00",USD,"2,634.00"
3,Trex_NYSE_TREX,200.00,200.00,"2,783.00",USD,"2,783.00"
4,ACM_Research_Inc,100.00,100.00,200.00,USD,200.00
5,Arcosa_NYSE_ACA,33.00,33.00,0.00,USD,0.00
6,Cigna_NYSE_CI,24.00,24.00,"7,601.00",USD,"7,601.00"
7,USD,1.70,1.70,1.70,USD,1.70


On the payment date, we can see that we hold 200 shares, we have received 100 shares. This is since the output factor was 2 in our data. Also note that we did not have to pay for these shares, so our cost amount remains unchanged.

In [397]:
get_portfolio_holdings("23/03/2022")

,properties.Instrument/default/Name.value.label_value,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,Apple_Nasdaq_AAPL,100.00,100.00,"4,375.00",USD,"4,375.00"
1,ExpressScripts_NYSE_ESRX,100.00,100.00,"7,601.00",USD,"7,601.00"
2,TrinityIndustries_NYSE_TRN,100.00,100.00,"2,634.00",USD,"2,634.00"
3,Trex_NYSE_TREX,200.00,200.00,"2,783.00",USD,"2,783.00"
4,ACM_Research_Inc,200.00,200.00,200.00,USD,200.00
5,Arcosa_NYSE_ACA,33.00,33.00,0.00,USD,0.00
6,Cigna_NYSE_CI,24.00,24.00,"7,601.00",USD,"7,601.00"
7,USD,1.70,1.70,1.70,USD,1.70
